In [ ]:
import PySimpleGUI as sg
import os
import pandas as pd
from docx import Document
import docx
import locale
import requests
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from docx import Document
from docx import * 
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches
import os
from datetime import datetime


# In[3]:


locale.setlocale(locale.LC_ALL, "cs_CZ.UTF-8");


# In[4]:

seznam_spolecnosti = pd.read_excel("adresy_a_informace.xlsx")
seznam_spolecnosti_vyber = list(seznam_spolecnosti["Název úvěru"])

# In[6]:


pracovni_slozka = os.getcwd()
layout = [
 [sg.Image(filename="Ronda1.png", size=(800,500))],   
 [sg.Text("Vyber excel soubor: ")],
 [sg.InputText(key="CESTA_SOUBORU"),
 sg.FileBrowse(button_text = "Vyhledat",initial_folder=pracovni_slozka, file_types = [("Excel Files", "*.xlsx")])],
 [sg.Text("Zadej počet měsíců první opce: "), sg.InputText(key="POCET_MESICU_PRVNI")],
 [sg.Text("Zadej počet měsíců druhé opce: "), sg.InputText(key="POCET_MESICU_DRUHA")],
 [sg.Text("Vyber datum: ")], [sg.In(key="-CAL-", enable_events=True, visible=False), sg.CalendarButton(button_text="Vybrat", locale="cs_CZ")],
 [sg.Text("Vyber společnost: "), sg.Combo(values=seznam_spolecnosti_vyber, key="SPOLECNOST")],
 [sg.Button("Vygenerovat", button_color=("white", "green")), sg.Exit(button_color=("white", "red"))]
]
sg.theme('SystemDefault1')
okno = sg.Window("Generátor vyčíslení úvěru").layout(layout)

while True:
    tlacitko, hodnoty = okno.read()
    if tlacitko in (sg.WIN_CLOSED, "Exit"):
        break
    
    elif tlacitko == "Vygenerovat":
        odpoved = hodnoty["SPOLECNOST"]
        cesta_souboru = hodnoty["CESTA_SOUBORU"]
        calendar = pd.read_excel(cesta_souboru)
        calendar = calendar.drop(calendar.index[0])
        mesic_prvni_opce = hodnoty["POCET_MESICU_PRVNI"]
        mesic_prvni_opce = int(mesic_prvni_opce)
        mesic_druhe_opce = hodnoty["POCET_MESICU_DRUHA"]
        mesic_druhe_opce = int(mesic_druhe_opce)
        datum_vycisleni = hodnoty["-CAL-"]
        datum_vycisleni = datetime.strptime(datum_vycisleni, '%Y-%m-%d %H:%M:%S')
        datum_vycisleni_upravene = datum_vycisleni.strftime('%d.%m.%Y')
        datum_vycisleni_upravene = datetime.strptime(datum_vycisleni_upravene, '%d.%m.%Y')
        

        # zjištění data čerpání
        calendar['Datum'] = pd.to_datetime(calendar['Datum'], format='%d.%m.%Y')
        datum_cerpani = calendar["Datum"].iloc[0]

        datum_prvni_opce = datum_cerpani + relativedelta(months=mesic_prvni_opce)
        finalni_datum = datum_prvni_opce
        finalni_datum = finalni_datum.to_pydatetime()
        if finalni_datum < datum_vycisleni_upravene:
            datum_druhe_opce = datum_prvni_opce
            datum_druhe_opce = datum_druhe_opce.to_pydatetime()
            while datum_druhe_opce < datum_vycisleni_upravene:
                datum_druhe_opce = datum_druhe_opce + relativedelta(months=mesic_druhe_opce)
                finalni_datum = datum_druhe_opce 
        finalni_datum.strftime("%d.%m.%Y")

        jistina = 0
        úroky = 0
        poplatky = 0
        mesic_a_rok_k_datu_vycisleni = datum_vycisleni_upravene.strftime("%m.%Y")

        # vyhledání řádku obsahujícího měsíc vyčíslení
        radek_roku_a_mesice_k_datu_vycisleni = calendar[calendar['Datum'].dt.strftime("%m.%Y") == mesic_a_rok_k_datu_vycisleni]
        pozice_mesice_vycisleni = radek_roku_a_mesice_k_datu_vycisleni.index[0]

        # pozice za opční měsíce
        opce_mesic_a_rok = finalni_datum.strftime("%m.%Y")
        opce_rok_a_mesic = calendar[calendar['Datum'].dt.strftime("%m.%Y") == opce_mesic_a_rok]
        pozice_opce_mesice = opce_rok_a_mesic.index[0]
       
        # vytvoření nového DataFrame obsahujícího řádky mezi těmito hranicemi
        calendar_vycisleni = calendar.iloc[(pozice_mesice_vycisleni-1): (pozice_opce_mesice)]
        
        # uroky

        uroky_vyciselni_bez_posledniho_mesice = calendar_vycisleni.loc[calendar_vycisleni.index[:-1], "Úrok"].sum()

        # poslední řádek upraveného df a zjištění počtu dnů v poslendím řádku
        posledni_radka = calendar_vycisleni["Datum"].iloc[-1]
        posledni_datum_den = posledni_radka.strftime("%d")


        den_z_finalniho_data = finalni_datum.strftime("%d")


        uroky_posledni_mesic = calendar_vycisleni.loc[calendar_vycisleni.index[-1], "Úrok"]/int(posledni_datum_den)*int(den_z_finalniho_data)
        urok_celkem = uroky_vyciselni_bez_posledniho_mesice + uroky_posledni_mesic

        # poplatky

        poplatky_celkem = calendar_vycisleni["Poplatek"].sum()

        # jistina

        # dataframe mezi následujícím měsícem a posledním měsícem splátkového kalendáře

        calendar_vycisleni_jistina = calendar.iloc[(pozice_mesice_vycisleni-1):]
        jistina_celkem = round(calendar_vycisleni_jistina["Úmor"].sum(),2)
        

        # celkem
        celkem_k_uhrade = jistina_celkem + poplatky_celkem + urok_celkem

        # Vytvoření wordového dokumentu
        doc = Document()

        
        section = doc.sections[0]
        header = section.header
        paragraph = header.paragraphs[0]
        run = paragraph.add_run()
        run.add_picture("Ronda1.png")

        footer = section.footer
 

        footer_para = footer.paragraphs[0]
 

        footer_para.text = "RONDA INVEST a.s.\n Sídlo: Výtvarná 1023/4, 161 00 Praha 6 – Ruzyně, tel.: +420 296 187 888, email: info@rondainvest.cz, web: www.rondainvest.cz IČO: 06370772, zapsaná v obchodním rejstříku vedeném u Městského soudu v Praze, oddíl B, vložka číslo 22788"

        radek_dle_vyberu_klienta = seznam_spolecnosti.loc[seznam_spolecnosti["Název úvěru"]==odpoved]
        klient = radek_dle_vyberu_klienta["Název"].to_string()
        klient = klient[5:]

        # Přidání hlavičky
        paragraph = doc.add_paragraph()
        run = paragraph.add_run(klient)
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run.font.name = "Verdana"
        run.font.size = (120000)
        run.bold = True

        ulice = radek_dle_vyberu_klienta["Adresa"].to_string().split(";")[0]
        ulice = ulice[5:]

        paragraph = doc.add_paragraph()
        run = paragraph.add_run(ulice)
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run.font.name = "Verdana"
        run.font.size = (120000)


        mesto = radek_dle_vyberu_klienta["Adresa"].to_string().split(";")[1]

        paragraph = doc.add_paragraph()
        run = paragraph.add_run(mesto)
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run.font.name = "Verdana"
        run.font.size = (120000)

        # Přidání datumu
        now = datetime.now()
        paragraph = doc.add_paragraph()
        run = paragraph.add_run('V Praze dne ' + now.strftime("%d.%m.%Y"))
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        
        datum_vycisleni_upravene = datum_vycisleni.strftime('%d.%m.%Y')

        # Přidání předmětu dopisu
        paragraph = doc.add_paragraph()
        run = paragraph.add_run("Vyčíslení úvěru k: " + datum_vycisleni_upravene)
        run.bold = True

        cislo_smlouvy = radek_dle_vyberu_klienta["Číslo smlouvy"].to_string()
        cislo_smlouvy = cislo_smlouvy[6:]

        # Přidání textu 
        datum_cerpani = datetime.strftime(datum_cerpani, "%d.%m.%Y")
        datum_cerapni = datetime.strptime(datum_cerpani, '%d.%m.%Y').date()
        paragraph = doc.add_paragraph()
        run = paragraph.add_run("Společnost Ronda Invest, a.s. IČ: 06370772, se sídlem: Výtvarná 1023/4, Ruzyně, 161 00 Praha 6 tímto vyjadřuje souhlas s  předčasným splacením úvěru č. " + str(cislo_smlouvy) + ", uzavřený s Vámi dne " + str(datum_cerpani) +".")


        # Přidání tabulky vyčíselní
    
        table = doc.add_table(rows=4, cols=2)
        table.style = 'Table Grid'
        table.autofit = True


        # Zajištění šířky levého sloupce
        left_column_cells = table.column_cells(0)
        for cell in left_column_cells:
            cell.width = Inches(10)

        formatovane_celkem_k_uhrade = locale.format_string("%.2f", celkem_k_uhrade, grouping=True)
        formatovane_jistina_celkem = locale.format_string("%.2f", jistina_celkem, grouping=True)
        formatovane_urok_celkem = locale.format_string("%.2f", urok_celkem, grouping=True)
        formatovane_poplatky_celkem = locale.format_string("%.2f", poplatky_celkem, grouping=True)

        table.cell(0, 0).text = "K datu " + str(datum_vycisleni_upravene) + " bude činit dluh přílušného úvěru."
        table.cell(0, 1).text = formatovane_celkem_k_uhrade + " Kč"
        table.cell(1, 0).text = "Nesplacená částka jistiny k " + str(datum_vycisleni_upravene)
        table.cell(1, 1).text = formatovane_jistina_celkem + " Kč"
        table.cell(2, 0).text = "Úroky za období od " + str(datum_vycisleni_upravene) + " - " + str(finalni_datum.strftime("%d.%m.%Y"))
        table.cell(2, 1).text = formatovane_urok_celkem + " Kč"
        table.cell(3, 0).text = "Poplatek za správu úvěru"
        table.cell(3, 1).text = formatovane_poplatky_celkem + " Kč"

        # Zajištění zarovnání pravého sloupce
        right_column_cells = table.column_cells(1)
        for cell in right_column_cells:
            for paragraph in cell.paragraphs:
                paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

        # Zajištění ztučnění prvního řádku
        for i in range(2):
            table.cell(0, i).paragraphs[0].runs[0].bold = True
        p = doc.add_paragraph("\nCelkem bude tedy nutné uhradit k " + str(datum_vycisleni_upravene) + " částku ve výši ")
        p.add_run(formatovane_celkem_k_uhrade).bold = True
        p.add_run(" Kč. Výše uvedené stavy jednotlivých částí pohledávek z předmětného úvěru byly vyčísleny v den zpracování tohoto dopisu")
        p.add_run(" předpokládají úhradu splátky úvěru k poslednímu dni tohoto měsíce").underline = True
        p.add_run(".")

        p = doc.add_paragraph()
        p.add_run("Prostředky zašlete na běžný účet ")
        p.add_run("č. 1139864 / 2060 vedený u Citfin, spořitelní družstvo s variabilním symbolem " + str(cislo_smlouvy) + " nejpozději do " + str(datum_vycisleni_upravene) + ".").bold = True

        paragraph = doc.add_paragraph()
        run = paragraph.add_run("Realizaci provedené úhrady dojde k plnému splacení úvěrů.")

        paragraph = doc.add_paragraph()
        run = paragraph.add_run("Toto Vyčíslení je platné za předpokladu úhrad všech řádných splátek dle splátkového kalendáře.")



        paragraph = doc.add_paragraph()
        run = paragraph.add_run("__________________________\n")
        run = paragraph.add_run("   Ing. Marek Košecký  \n")
        run = paragraph.add_run("    člen správní rady   ")

        konec_smlouvy = cislo_smlouvy[-4:]

        doc.save("vycisleni" + str(konec_smlouvy) +".docx")

okno.close()


# In[37]:


import locale

locale.setlocale(locale.LC_ALL, 'cs_CZ.UTF-8')
number = 1234567.89
formatted_number = locale.format_string("%.2f", number, grouping=True)
print(formatted_number)

